In [75]:
from rdflib import Graph, URIRef
from mcs_formalizations.namespace import bind_namespaces, MCS, RDF
from mcs_formalizations.helpers import uri_base

### Question: True/False - A parking garage is larger than an automobile.

##### Prepare the Graph

In [76]:
from mcs_formalizations.reasoners.sparql_reasoner import SparqlReasoner
parking_garage = uri_base("parking_garage")
automobile = uri_base("automobile")
vehicle = uri_base("vehicle")

##### Minimum Knowledge Assumed:
- a parking garage contains vehicles
- an automobile is a type of vehicle

In [77]:
reasoner = SparqlReasoner()

reasoner.add_knowledge(
    (parking_garage, MCS.contains, vehicle),
    (vehicle, RDF.type, automobile),
)

TypeError: add_knowledge() takes 2 positional arguments but 3 were given

#### Minimum rules assumed:

- a container is larger than the things it contains

In [ ]:
reasoner.apply_rules(
    "is_larger_than",
)

In [ ]:
print("Is a parking garage larger than an automobile?")
print(reasoner.ask((parking_garage, uri_base("isLargerThan"), automobile)))